In [4]:
import numpy as np
import typing
from typing import Any, Tuple
import tensorflow as tf
import tensorflow_text as tf_text
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
use_builtins = True

Shape checker

PREPARE DATA

In [5]:
def load_data(path_to_file):
    file = open(path_to_file,'r').read()
    qna_list = [f.split('\t') for f in file.split('\n')]
    x = [x[0] for x in qna_list]
    y = [y[1] for y in qna_list]
 
    return x, y

In [6]:
x, y = load_data('D:/chatbot_project/archive/dialogs.txt')

CREATE A TF.DATA DATASET

In [7]:
BUFFER_SIZE = len(x)
BATCH_SIZE = 64
TRAIN_SIZE = int(0.8 * BUFFER_SIZE/BATCH_SIZE)
TEST_SIZE = int(0.2 * BUFFER_SIZE/BATCH_SIZE)

dataset = tf.data.Dataset.from_tensor_slices((x, y)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_data = dataset.take(TRAIN_SIZE)
test_data = dataset.skip(TRAIN_SIZE)
# val_data = test_data.skip(VAL_SIZE)
# test_data = test_data.take(TEST_SIZE)

In [8]:
# for example_input_batch, example_target_batch in dataset.take(1):
#   print(example_input_batch[:5])
#   print()
#   print(example_target_batch[:5])
#   break

TEXT PROCESSING 

standarization

In [9]:
# example_text = tf.constant('¿Todavía está en casa?')

# print(example_text.numpy())
# print(tf_text.normalize_utf8(example_text, 'NFKD').numpy())

In [10]:
def tf_lower_and_split_punct(text):
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  text = tf.strings.strip(text)
  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

In [11]:
# print(example_text.numpy().decode())
# print(tf_lower_and_split_punct(example_text).numpy().decode())

text vectorization

In [12]:
max_vocab_size = 5000

input_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size)

In [14]:
input_text_processor.adapt(x)
print(input_text_processor)

# # Here are the first 10 words from the vocabulary:
# input_text_processor.get_vocabulary()[:10]

In [16]:
output_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size)

output_text_processor.adapt(y)
# output_text_processor.get_vocabulary()[:10]

In [17]:
# example_tokens = input_text_processor(example_input_batch)
# example_tokens[:3, :10]

In [18]:
# input_vocab = np.array(input_text_processor.get_vocabulary())
# tokens = input_vocab[example_tokens[0].numpy()]
# ' '.join(tokens)

In [19]:
# plt.subplot(1, 2, 1)
# plt.pcolormesh(example_tokens)
# plt.title('Token IDs')

# plt.subplot(1, 2, 2)
# plt.pcolormesh(example_tokens != 0)
# plt.title('Mask')

In [20]:
# defining constants for the model 
embedding_dim = 256
units = 1024

In [21]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, input_vocab_size, embedding_dim, enc_units):
    super(Encoder, self).__init__()
    self.enc_units = enc_units
    self.input_vocab_size = input_vocab_size

    # The embedding layer converts tokens to vectors
    self.embedding = tf.keras.layers.Embedding(self.input_vocab_size,
                                               embedding_dim)

    # The GRU RNN layer processes those vectors sequentially.
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   # Return the sequence and state
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, tokens, state=None):
    # 2. The embedding layer looks up the embedding for each token.
    vectors = self.embedding(tokens)
    # 3. The GRU processes the embedding sequence.
    #    output shape: (batch, s, enc_units)
    #    state shape: (batch, enc_units)
    output, state = self.gru(vectors, initial_state=state)
    # 4. Returns the new sequence and its state.
    return output, state

In [22]:
# # Convert the input text to tokens.
# example_tokens = input_text_processor(example_input_batch)

# Encode the input sequence.
encoder = Encoder(input_text_processor.vocabulary_size(),
                  embedding_dim, units)
# example_enc_output, example_enc_state = encoder(example_tokens)

# print(f'Input batch, shape (batch): {example_input_batch.shape}')
# print(f'Input batch tokens, shape (batch, s): {example_tokens.shape}')
# print(f'Encoder output, shape (batch, s, units): {example_enc_output.shape}')
# print(f'Encoder state, shape (batch, units): {example_enc_state.shape}')

In [23]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super().__init__()
    # score(ht, hs) = 
    # Luong's multiplicative style: ht.transpose*W*hs
    # Bahdanau's additive style: Va.transpose* tanh(W1*ht+W2*hs)
    self.W1 = tf.keras.layers.Dense(units, use_bias=False)
    self.W2 = tf.keras.layers.Dense(units, use_bias=False)

    self.attention = tf.keras.layers.AdditiveAttention()

  def call(self, query, value, mask):
    # W1*ht
    w1_query = self.W1(query)

    # W2*hs
    w2_key = self.W2(value)

    query_mask = tf.ones(tf.shape(query)[:-1], dtype=bool)
    value_mask = mask

    context_vector, attention_weights = self.attention(
        inputs = [w1_query, value, w2_key],
        mask=[query_mask, value_mask],
        return_attention_scores = True,
    )


    return context_vector, attention_weights

In [24]:
attention_layer = BahdanauAttention(units)

In [25]:
# (example_tokens != 0).shape

In [26]:
# # Later, the decoder will generate this attention query
# example_attention_query = tf.random.normal(shape=[len(example_tokens), 2, 10])

# # Attend to the encoded tokens

# context_vector, attention_weights = attention_layer(
#     query=example_attention_query,
#     value=example_enc_output,
#     mask=(example_tokens != 0))

# print(f'Attention result shape: (batch_size, query_seq_length, units):           {context_vector.shape}')
# print(f'Attention weights shape: (batch_size, query_seq_length, value_seq_length): {attention_weights.shape}')

In [27]:
# plt.subplot(1, 2, 1)
# plt.pcolormesh(attention_weights[:, 0, :])
# plt.title('Attention weights')

# plt.subplot(1, 2, 2)
# plt.pcolormesh(example_tokens != 0)
# plt.title('Mask')

In [28]:
# attention_weights.shape
class DecoderInput(typing.NamedTuple):
  new_tokens: Any
  enc_output: Any
  mask: Any

class DecoderOutput(typing.NamedTuple):
  logits: Any
  attention_weights: Any

In [29]:
# attention_slice = attention_weights[0, 0].numpy()
# attention_slice = attention_slice[attention_slice != 0]

In [30]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, output_vocab_size, embedding_dim, dec_units):
    super(Decoder, self).__init__()
    self.dec_units = dec_units
    self.output_vocab_size = output_vocab_size
    self.embedding_dim = embedding_dim

    # For Step 1. The embedding layer convets token IDs to vectors
    self.embedding = tf.keras.layers.Embedding(self.output_vocab_size,
                                               embedding_dim)

    # For Step 2. The RNN keeps track of what's been generated so far.
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

    # For step 3. The RNN output will be the query for the attention layer.
    self.attention = BahdanauAttention(self.dec_units)

    # For step 4. Eqn. (3): converting `ct` to `at`
    self.Wc = tf.keras.layers.Dense(dec_units, activation=tf.math.tanh,
                                    use_bias=False)

    # For step 5. This fully connected layer produces the logits for each
    # output token.
    self.fc = tf.keras.layers.Dense(self.output_vocab_size)

  def call(self, inputs: DecoderInput, state=None) -> Tuple[DecoderOutput, tf.Tensor]:
    vectors = self.embedding(inputs.new_tokens)
    rnn_output, state = self.gru(vectors, initial_state=state)
    context_vector, attention_weights = self.attention(query=rnn_output, value=inputs.enc_output, mask=inputs.mask)
    context_and_rnn_output = tf.concat([context_vector, rnn_output], axis=-1)
    attention_vector = self.Wc(context_and_rnn_output)
    logits = self.fc(attention_vector)

    return DecoderOutput(logits, attention_weights), state

In [31]:
# Decoder.call = call

In [32]:
decoder = Decoder(output_text_processor.vocabulary_size(),
                  embedding_dim, units)

In [33]:
# # Convert the target sequence, and collect the "[START]" tokens
# example_output_tokens = output_text_processor(example_target_batch)

# start_index = output_text_processor.get_vocabulary().index('[START]')
# first_token = tf.constant([[start_index]] * example_output_tokens.shape[0])

In [34]:
# # Run the decoder
# dec_result, dec_state = decoder(
#     inputs = DecoderInput(new_tokens=first_token,
#                           enc_output=example_enc_output,
#                           mask=(example_tokens != 0)),
#     state = example_enc_state
# )

# print(f'logits shape: (batch_size, t, output_vocab_size) {dec_result.logits.shape}')
# print(f'state shape: (batch_size, dec_units) {dec_state.shape}')

In [35]:
# sampled_token = tf.random.categorical(dec_result.logits[:, 0, :], num_samples=1)

In [36]:
# vocab = np.array(output_text_processor.get_vocabulary())
# first_word = vocab[sampled_token.numpy()]
# first_word[:5]

LOSS FUNCTION 

In [37]:
class MaskedLoss(tf.keras.losses.Loss):
  def __init__(self):
    self.name = 'masked_loss'
    self.loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')

  def __call__(self, y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss = self.loss(y_true, y_pred)
    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, tf.float32)
    loss *= mask
    # Return the total.
    return tf.reduce_sum(loss)

TRAINING STEP 

In [38]:
class TrainChatbot(tf.keras.Model):
  def __init__(self, embedding_dim, units,
               input_text_processor,
               output_text_processor, 
               use_tf_function=True):
    super().__init__()
    encoder = Encoder(input_text_processor.vocabulary_size(), embedding_dim, units)
    decoder = Decoder(output_text_processor.vocabulary_size(), embedding_dim, units)

    self.encoder = encoder
    self.decoder = decoder
    self.input_text_processor = input_text_processor
    self.output_text_processor = output_text_processor
    self.use_tf_function = use_tf_function

  def train_step(self, inputs):
    if self.use_tf_function:
      return self._tf_train_step(inputs)
    else:
      return self._train_step(inputs)
      
  def _preprocess(self, input_text, target_text):
    input_tokens = self.input_text_processor(input_text)
    target_tokens = self.output_text_processor(target_text)
    input_mask = input_tokens != 0
    target_mask = target_tokens != 0
    return input_tokens, input_mask, target_tokens, target_mask

  def _train_step(self, inputs):
    input_text, target_text = inputs  
    (input_tokens, input_mask, target_tokens, target_mask) = self._preprocess(input_text, target_text)
    max_target_length = tf.shape(target_tokens)[1]

    with tf.GradientTape() as tape:
      enc_output, enc_state = self.encoder(input_tokens)
      dec_state = enc_state
      loss = tf.constant(0.0)

      for t in tf.range(max_target_length-1):
        new_tokens = target_tokens[:, t:t+2]
        step_loss, dec_state = self._loop_step(new_tokens, input_mask,
                                              enc_output, dec_state)
        loss = loss + step_loss
      average_loss = loss / tf.reduce_sum(tf.cast(target_mask, tf.float32))
    variables = self.trainable_variables 
    gradients = tape.gradient(average_loss, variables)
    self.optimizer.apply_gradients(zip(gradients, variables))
    return {'batch_loss': average_loss}
  
  def _loop_step(self, new_tokens, input_mask, enc_output, dec_state):
    input_token, target_token = new_tokens[:, 0:1], new_tokens[:, 1:2]
    decoder_input = DecoderInput(new_tokens=input_token, enc_output=enc_output, mask=input_mask)

    dec_result, dec_state = self.decoder(decoder_input, state=dec_state)
    y = target_token
    y_pred = dec_result.logits
    step_loss = self.loss(y, y_pred)

    return step_loss, dec_state
    
  @tf.function(input_signature=[[tf.TensorSpec(dtype=tf.string, shape=[None]), tf.TensorSpec(dtype=tf.string, shape=[None])]])
  def _tf_train_step(self, inputs):
      return self._train_step(inputs)


In [39]:
# TrainChatbot._preprocess = _preprocess

In [40]:
# TrainChatbot._train_step = _train_step

In [41]:
# TrainChatbot._loop_step = _loop_step

In [42]:
chatbot = TrainChatbot(
    embedding_dim, units,
    input_text_processor=input_text_processor,
    output_text_processor=output_text_processor,
    use_tf_function=False)

# Configure the loss and optimizer
chatbot.compile(
    optimizer=tf.optimizers.Adam(),
    loss=MaskedLoss(),
)

In [43]:
# TrainChatbot._tf_train_step = _tf_train_step
chatbot.use_tf_function = True

In [44]:
train_chatbot = TrainChatbot(
    embedding_dim, units,
    input_text_processor=input_text_processor,
    output_text_processor=output_text_processor)


train_chatbot.compile(
    optimizer=tf.optimizers.Adam(),
    loss=MaskedLoss(),
)

In [45]:
class BatchLogs(tf.keras.callbacks.Callback):
  def __init__(self, key):
    self.key = key
    self.logs = []

  def on_train_batch_end(self, n, logs):
    self.logs.append(logs[self.key])

batch_loss = BatchLogs('batch_loss')

In [51]:
train_chatbot.fit(train_data, epochs=2, callbacks=[batch_loss])

Epoch 1/2
46/46 [==============================] - 144s 3s/step - batch_loss: 4.1654
Epoch 2/2
46/46 [==============================] - 146s 3s/step - batch_loss: 3.8648


In [ ]:

class Chatbot(tf.Module):
    def __init__(self, encoder, decoder, input_text_processor, output_text_processor):
        self.encoder = encoder
        self.decoder = decoder
        self.input_text_processor = input_text_processor
        self.output_text_processor = output_text_processor

        self.output_token_string_from_index = (tf.keras.layers.StringLookup(vocabulary=output_text_processor.get_vocabulary(), mask_token='', invert=True))

        index_from_string = tf.keras.layers.StringLookup(vocabulary=output_text_processor.get_vocabulary(), mask_token='')
        token_mask_ids = index_from_string(['', '[UNK]', '[START]']).numpy()

        token_mask = np.zeros([index_from_string.vocabulary_size()], dtype=np.bool)
        token_mask[np.array(token_mask_ids)] = True
        self.token_mask = token_mask

        self.start_token = index_from_string(tf.constant('[START]'))
        self.end_token = index_from_string(tf.constant('[END]'))

    def tokens_to_text(self, result_tokens):
        result_text_tokens = self.output_token_string_from_index(result_tokens)
        result_text = tf.strings.reduce_join(result_text_tokens, axis=1, separator=' ')
        result_text = tf.strings.strip(result_text)
        return result_text

    def sample(self, logits, temperature):
        token_mask = self.token_mask[tf.newaxis, tf.newaxis, :]
        logits = tf.where(self.token_mask, -np.inf, logits)

        if temperature == 0.0:
            new_tokens = tf.argmax(logits, axis=-1)
        else: 
            logits = tf.squeeze(logits, axis=1)
            new_tokens = tf.random.categorical(logits/temperature, num_samples=1)
        return new_tokens

    def chatbot_unrolled(self, input_text, *, max_length=50, return_attention=True, temperature=1.0):
        batch_size = tf.shape(input_text)[0]
        input_tokens = self.input_text_processor(input_text)
        enc_output, enc_state = self.encoder(input_tokens)

        dec_state = enc_state
        new_tokens = tf.fill([batch_size, 1], self.start_token)

        result_tokens = []
        attention = []
        done = tf.zeros([batch_size, 1], dtype=tf.bool)

        for _ in range(max_length):
            dec_input = DecoderInput(new_tokens=new_tokens, enc_output=enc_output, mask=(input_tokens!=0))
            dec_result, dec_state = self.decoder(dec_input, state=dec_state)
            attention.append(dec_result.attention_weights)
            new_tokens = self.sample(dec_result.logits, temperature)
            done = done | (new_tokens == self.end_token)
            new_tokens = tf.where(done, tf.constant(0, dtype=tf.int64), new_tokens)
            result_tokens.append(new_tokens)

            if tf.executing_eagerly() and tf.reduce_all(done):
                break

        result_tokens = tf.concat(result_tokens, axis=-1)
        result_text = self.tokens_to_text(result_tokens)

        if return_attention:
            attention_stack = tf.concat(attention, axis=1)
            return {'text': result_text, 'attention': attention_stack}
        else:
            return {'text': result_text}
    
    @tf.function(input_signature=[tf.TensorSpec(dtype=tf.string, shape=[None])])
    def tf_translate(self, input_text):
        return self.translate(input_text)

In [52]:
%%time
input_text = tf.constant([
    'hi, how are you doing?', # "It's really cold here."
    "i'm fine. how about yourself?"
])

result = chatbot.translate(
    input_text = input_text)

print(result['text'][0].numpy().decode())
print(result['text'][1].numpy().decode())
print()